In [1]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [10]:
token_for_foursquare = os.getenv("API_Key")

In [7]:
query = "cafetería"

In [55]:
def get_results_from_foursquare (query, location, limit=1):
    
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_for_foursquare,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

In [56]:
location = [41.3977777, 2.1882621]

In [57]:
query = "cafeteria"

In [58]:
response_function = get_results_from_foursquare(query, location )

In [64]:
response_function["results"][0]

{'fsq_id': '619b423c52dcb37fe9fa5347',
 'categories': [{'id': 13034,
   'name': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 1164,
 'geocodes': {'main': {'latitude': 41.408225, 'longitude': 2.187253}},
 'link': '/v3/places/619b423c52dcb37fe9fa5347',
 'location': {'address': 'Avinguda Meridiana, 116-120',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': '',
  'formatted_address': 'Avinguda Meridiana, 116-120, 08018 Barcelona Catalonia',
  'locality': 'Barcelona',
  'postcode': '08018',
  'region': 'Catalonia'},
 'name': 'Cafeteria 365',
 'related_places': {},
 'timezone': 'Europe/Madrid'}